In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc, confusion_matrix
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import json

import tf2onnx
import os


### А тут время интересных историй
Я как вроде умный человек потыкался в момент выполнения кода, сколько же задействуестья ядер (ну скорее потоков, но не будем душить) процессора оказалось что всего одно. Мне стало очень грусно, и я подумал а что если обучать на встроенном видео ядре, оно же сможет распаралелить и оооочень быстро посчитать, на проце условная моделька считалась 45 секунд, я разобрался и смог запустить GPU (встройка в m1)


In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


После этого стало всего 19, но минут... Эхххх https://www.youtube.com/watch?v=zKwYz83DFKY

Ну ладно, если будет время, я ещё попробую на другом пк с дискретной картой. Много часов спустя, я могу сказать что я несправился

Ещё была идея взять нормальный человеческий гугл колаб, но чет я зажал 10 USD

Да да да, это всё избыточно, я поэтосамовал на такие идеи примерно 6 часов, но как говорит мой друг, это не ~~военное~~ преступление если тебе было весело. Мне было весело

In [ ]:
tf.random.set_seed(43)

In [ ]:
data = pd.read_csv("training.csv.gz")

In [ ]:
data.head(15)

### Список колонок в выборках

Здесь **Spd** обозначает Scintillating Pad Detector (сцинтилляционный детектор), **Prs** - Preshower (предшоуэрный детектор), **Ecal** - электромагнитный калориметр, **Hcal** - адронный калориметр, **Brem** обозначает следы частиц, отклоненных детектором.

- ID - идентификатор для треков (присутствует только в тестовом файле для целей отправки)
- Label - строковое значение, обозначающее типы частиц. Может принимать значения "Electron", "Muon", "Kaon", "Proton", "Pion" и "Ghost". Эта колонка отсутствует в тестовом файле.
- FlagSpd - флаг (0 или 1), если реконструированный трек проходит через Spd
- FlagPrs - флаг (0 или 1), если реконструированный трек проходит через Prs
- FlagBrem - флаг (0 или 1), если реконструированный трек проходит через Brem
- FlagEcal - флаг (0 или 1), если реконструированный трек проходит через Ecal
- FlagHcal - флаг (0 или 1), если реконструированный трек проходит через Hcal
- FlagRICH1 - флаг (0 или 1), если реконструированный трек проходит через первый детектор RICH
- FlagRICH2 - флаг (0 или 1), если реконструированный трек проходит через второй детектор RICH
- FlagMuon - флаг (0 или 1), если реконструированный трек проходит через мюонные станции (Muon)
- SpdE - энергия, связанная с треком в Spd
- PrsE - энергия, связанная с треком в Prs
- EcalE - энергия, связанная с треком в Ecal
- HcalE - энергия, связанная с треком в Hcal
- PrsDLLbeElectron - дельта логарифма правдоподобия для кандидата в частицы быть электроном, используя информацию из Prs
- BremDLLbeElectron - дельта логарифма правдоподобия для кандидата в частицы быть электроном, используя информацию из Brem
- TrackP - импульс частицы
- TrackPt - поперечный импульс частицы
- TrackNDoFSubdetector1 - число степеней свободы для подгонки трека, используя попадания в субдетектор1
- TrackQualitySubdetector1 - качество подгонки трека (chi2), используя попадания в субдетектор1
- TrackNDoFSubdetector2 - число степеней свободы для подгонки трека, используя попадания в субдетектор2
- TrackQualitySubdetector2 - качество подгонки трека (chi2), используя попадания в субдетектор2
- TrackNDoF - число степеней свободы для подгонки трека, используя попадания во все субдетекторы
- TrackQualityPerNDoF - качество подгонки трека (chi2) на степень свободы
- TrackDistanceToZ - расстояние между треком и осью z (осью пучка)
- Calo2dFitQuality - качество 2D подгонки кластеров в калориметре
- Calo3dFitQuality - качество 3D подгонки в калориметре с предположением, что частица была электроном
- EcalDLLbeElectron - дельта логарифма правдоподобия для кандидата в частицы быть электроном, используя информацию из Ecal
- EcalDLLbeMuon - дельта логарифма правдоподобия для кандидата в частицы быть мюоном, используя информацию из Ecal
- EcalShowerLongitudinalParameter - продольный параметр шауэра в Ecal
- HcalDLLbeElectron - дельта логарифма правдоподобия для кандидата в частицы быть электроном, используя информацию из Hcal
- HcalDLLbeMuon - дельта логарифма правдоподобия для кандидата в частицы быть мюоном, используя информацию из Hcal
- RICHpFlagElectron - флаг (0 или 1), если импульс больше порога для электронов, чтобы произвести свет Черенкова
- RICHpFlagProton - флаг (0 или 1), если импульс больше порога для протонов, чтобы произвести свет Черенкова
- RICHpFlagPion - флаг (0 или 1), если импульс больше порога для пионов, чтобы произвести свет Черенкова
- RICHpFlagKaon - флаг (0 или 1), если импульс больше порога для каонов, чтобы произвести свет Черенкова
- RICHpFlagMuon - флаг (0 или 1), если импульс больше порога для мюонов, чтобы произвести свет Черенкова
- RICH_DLLbeBCK - дельта логарифма правдоподобия для кандидата в частицы быть фоновым, используя информацию из RICH
- RICH_DLLbeKaon - дельта логарифма правдоподобия для кандидата в частицы быть каоном, используя информацию из RICH
- RICH_DLLbeElectron - дельта логарифма правдоподобия для кандидата в частицы быть электроном, используя информацию из RICH
- RICH_DLLbeMuon - дельта логарифма правдоподобия для кандидата в частицы быть мюоном, используя информацию из RICH
- RICH_DLLbeProton - дельта логарифма правдоподобия для кандидата в частицы быть протоном, используя информацию из RICH
- MuonFlag - мюонный флаг (является ли этот трек мюоном), который определяется по мюонным станциям
- MuonLooseFlag - мюонный флаг (является ли этот трек мюоном), который определяется по мюонным станциям с использованием более слабых критериев
- MuonLLbeBCK - логарифм правдоподобия для кандидата в частицы не быть мюоном, используя информацию из мюонных станций
- MuonLLbeMuon - логарифм правдоподобия для кандидата в частицы быть мюоном, используя информацию из мюонных станций
- DLLelectron - дельта логарифма правдоподобия для кандидата в частицы быть электроном, используя информацию из всех субдетекторов
- DLLmuon - дельта логарифма правдоподобия для кандидата в частицы быть мюоном, используя информацию из всех субдетекторов
- DLLkaon - дельта логарифма правдоподобия для кандидата в частицы быть каоном, используя информацию из всех субдетекторов
- DLLproton - дельта логарифма правдоподобия для кандидата в частицы быть протоном, используя информацию из всех субдетекторов
- GhostProbability - вероятность для кандидата в частицы быть призрачным треком. Эта переменная является выходом классификационной модели, используемой в алгоритме трекинга.

Дельта логарифма правдоподобия в описаниях признаков означает разницу между логарифмом правдоподобия для гипотезы массы, что данный трек создан некоторой частицей (например, электроном) и логарифмом правдоподобия для гипотезы массы, что данный трек создан пионом (то есть DLLpion = 0, и поэтому у нас нет этих колонок). Это сделано потому, что большинство треков (~80%) оставлены пионами, и на практике нам действительно нужно различать другие частицы от пионов. Иными словами, нулевая гипотеза заключается в том, что частица является пионом.

In [ ]:
x = np.asarray(data.loc[:, data.columns != 'Label'], dtype=np.float32)
le = preprocessing.LabelEncoder()
le.fit(data.Label)
y = le.transform(data.Label).astype(np.float32)
labels = np.array(le.classes_, dtype=str)

In [ ]:
print(x.shape, y.shape)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [ ]:
print(x_train.shape, y_train.shape)


In [ ]:
count = np.array([(data.Label == l).sum() for l in labels])
plt.bar(np.arange(len(labels)), count / np.sum(count))
plt.xticks(np.arange(len(labels)), labels)
plt.show()

Классы сбалансированы, ура

In [ ]:
INPUT_DIM  = x_train.shape[1]
HIDDEN_DIM = 100 
# ну вообще, HIDDEN_DIM это тоже гиперпараметр, и его тоже можно подбирать
# Например по кросс-валидации, но опять же, это избыточно, потому что AUC-ROC 0.97 и так можно получить
OUTPUT_DIM = len(labels)

# Ну а INPUT_DIM и OUTPUT_DIM это очевидно, размеры входа и выхода, что константа

In [ ]:
def create_model(input_dim, hidden_dim, output_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(hidden_dim, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        tf.keras.layers.Dense(hidden_dim, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        tf.keras.layers.Dense(hidden_dim // 2, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        tf.keras.layers.Dense(output_dim, activation='softmax', name='output')
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['AUC', 'accuracy'])
    
    return model


In [ ]:
model = create_model(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM)
model.summary()

In [ ]:
learning_rate = 1e-4
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, tf.keras.utils.to_categorical(y_train, num_classes=len(labels)),
        epochs=10, batch_size=16,
        validation_data=(x_test, tf.keras.utils.to_categorical(y_test, num_classes=len(labels))))


In [ ]:
spec = (tf.TensorSpec((None, INPUT_DIM), tf.float32, name="input"),)
output_path = "particles.onnx"
model.output_names = ['output'] # добавим такой костыль, чтобы tf2onnx не ругался, а то чет хз
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, output_path=output_path)

In [ ]:
def plot_roc_curve(y_test, y_score, labels,saving=True):
    fpr = {}
    tpr = {}
    roc_auc = {}
    
    plt.figure()
    for i, label in enumerate(labels):
        fpr[label], tpr[label], _ = roc_curve(y_test[:, i], y_score[:, i])
        roc_auc[label] = auc(fpr[label], tpr[label])
        plt.plot(fpr[label], tpr[label], lw=2, label=f'{label} (AUC = {roc_auc[label]:.4f})')

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    if saving:
        plt.savefig('roc.png')
    plt.show()
    plt.close()

    return roc_auc

In [ ]:
def plot_training_history(history):
    plt.figure(figsize=(10, 8))
    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.savefig('train.png')
    plt.show()
    plt.close()


In [ ]:
def plot_confusion_matrix(y_true, y_pred, labels):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=labels, yticklabels=labels, cmap=plt.cm.Blues)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.savefig('confusion.png')
    plt.show()
    plt.close()

In [ ]:
def save_metrics_to_json(metrics, filename='particles.json'):
    data = {"test_auc": metrics}
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=4)

In [ ]:
plot_training_history(history)

Ну в целом моделькак хорошая, как я понимаю даже на малом количестве шагов (шага 4) она даст норм результат

In [ ]:
y_test_cat = tf.keras.utils.to_categorical(y_test, num_classes=len(labels))
y_score = model.predict(x_test)
y_pred = np.argmax(y_score, axis=1)

plot_confusion_matrix(y_test, y_pred, labels)

In [ ]:
roc_auc = plot_roc_curve(y_test_cat, y_score, labels)

In [ ]:
save_metrics_to_json(roc_auc)

Давайте убедимся что всё открывается

In [ ]:
import onnxruntime as rt

Я что-то добавил новый файл, и оперативке поплохело, ну давайте уберемся

In [ ]:
import gc

In [ ]:
del data
del history
del x_train, x_test, y_train, y_test
gc.collect() # вероятно есть какой-то более умпный способ освободить память, но пока и так работает

In [ ]:
data_test = pd.read_csv("test.csv", index_col="ID")

In [ ]:
x_test = np.asarray(data_test, dtype=np.float32)
sess = rt.InferenceSession("particles.onnx")

input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

print("Inputs: {}".format([x.name for x in sess.get_inputs()]))
print("Outputs: {}".format([x.name for x in sess.get_outputs()]))

pred = sess.run([label_name], {input_name: x})
np.savetxt("predict.csv", pred[0], delimiter=",")


In [ ]:
print(pred)

ЭЭЭЭ ну в целом, это похоже на то что стоило бы ожидать от модельки

Ну в целом всё, можно было сделать и лучше, ещё поиграться с моделями, попробовать всё таки запустить мою декстопную 1660 s, поиграться с Yandex Compute Cloud, там у них есть по описанию очень не плохое решение https://yandex.cloud/ru/docs/tutorials/testing/hpc-on-preemptible, ну и как я понял они халявные 4 к дают за регу + поминутная тарификация в общем чиллллллл. Да для нашей задачи это избыточно избыточно избыточно, но просто интересно. И да на самом деле, сама такска решилась за 2 часа, но на всякие штуки вокруг, я потратил ещё часов 15... но с таким интресом, в общем респект за такую дызышку, не иронична самая забавная в курсе. Ладно, я что-то тут распинаюсь а дедлайн прошел уде 11 часов назад, но переносов ещё много так что пофиг.

Ну а что касаемо pytorch, ну тоже сделаю, раз уж начал, и в это потыкаюсь, но так как ранний дейлик не действует, то до следующей недели запушу.
Ладно всё я наверное закалебал своими тирадами с опечатками, так что закончим, бб <3